In [1]:
import polars as pl
import polars.selectors as cs

import altair as alt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_path = r'/Users/zygimantas/Downloads/computer_prices_all.csv'

In [3]:
df = pl.read_csv(df_path)

In [5]:
df.describe()

statistic,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,cpu_threads,cpu_base_ghz,cpu_boost_ghz,gpu_brand,gpu_model,gpu_tier,vram_gb,ram_gb,storage_type,storage_gb,storage_drive_count,display_type,display_size_in,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
str,str,str,str,f64,str,str,str,str,f64,f64,f64,f64,f64,str,str,f64,f64,f64,str,f64,f64,str,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64
"""count""","""100000""","""100000""","""100000""",100000.0,"""100000""","""100000""","""100000""","""100000""",100000.0,100000.0,100000.0,100000.0,100000.0,"""100000""","""100000""",100000.0,100000.0,100000.0,"""100000""",100000.0,100000.0,"""100000""",100000.0,"""100000""",100000.0,100000.0,100000.0,100000.0,"""100000""",100000.0,100000.0,100000.0,100000.0
"""null_count""","""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""",0.0,0.0,0.0,"""0""",0.0,0.0,"""0""",0.0,"""0""",0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",null,null,null,2022.32085,null,null,null,null,3.15349,10.51574,19.3727,2.591322,3.53131,null,null,2.99135,6.15218,39.7064,null,903.936,1.52498,null,20.126655,null,98.46486,41.81347,61.38345,272.5205,null,5.084764,4.2896992,22.20036,1928.76422
"""std""",null,null,null,2.025761,null,null,null,null,1.373175,5.044092,9.718426,0.336435,0.350024,null,null,1.459643,3.964926,31.902684,null,774.243654,0.797284,null,6.709577,null,43.301652,35.868841,62.795034,354.686355,null,0.245977,3.814628,10.2319,580.492689
"""min""","""Desktop""","""ASUS""","""ASUS Air 030""",2018.0,"""ChromeOS""","""2-in-1""","""AMD""","""AMD Ryzen 3 3102""",1.0,4.0,4.0,2.0,2.8,"""AMD""","""Apple Integrated""",1.0,0.0,8.0,"""HDD""",256.0,1.0,"""IPS""",13.3,"""1920x1080""",60.0,0.0,0.0,0.0,"""Wi-Fi 5""",4.2,0.92,12.0,372.99
"""25%""",null,null,null,2021.0,null,null,null,null,2.0,6.0,12.0,2.4,3.3,null,null,2.0,4.0,16.0,null,512.0,1.0,null,14.0,null,60.0,0.0,0.0,0.0,null,5.0,1.5,12.0,1503.99
"""50%""",null,null,null,2023.0,null,null,null,null,3.0,8.0,16.0,2.6,3.5,null,null,3.0,6.0,32.0,null,512.0,1.0,null,16.0,null,90.0,56.0,65.0,0.0,null,5.1,2.0,24.0,1863.99
"""75%""",null,null,null,2024.0,null,null,null,null,4.0,14.0,24.0,2.8,3.8,null,null,4.0,8.0,64.0,null,1024.0,2.0,null,27.0,null,120.0,70.0,90.0,650.0,null,5.2,7.0,24.0,2287.99
"""max""","""Laptop""","""Samsung""","""Samsung Zen ZX2""",2025.0,"""macOS""","""Workstation""","""Intel""","""Intel i9-14997""",6.0,28.0,56.0,3.4,4.5,"""NVIDIA""","""RX 7000 90""",6.0,16.0,144.0,"""SSD""",4096.0,4.0,"""VA""",34.0,"""3840x2160""",240.0,99.0,240.0,1200.0,"""Wi-Fi 7""",5.3,16.0,48.0,10984.99


In [6]:
df.collect_schema()

Schema([('device_type', String),
        ('brand', String),
        ('model', String),
        ('release_year', Int64),
        ('os', String),
        ('form_factor', String),
        ('cpu_brand', String),
        ('cpu_model', String),
        ('cpu_tier', Int64),
        ('cpu_cores', Int64),
        ('cpu_threads', Int64),
        ('cpu_base_ghz', Float64),
        ('cpu_boost_ghz', Float64),
        ('gpu_brand', String),
        ('gpu_model', String),
        ('gpu_tier', Int64),
        ('vram_gb', Int64),
        ('ram_gb', Int64),
        ('storage_type', String),
        ('storage_gb', Int64),
        ('storage_drive_count', Int64),
        ('display_type', String),
        ('display_size_in', Float64),
        ('resolution', String),
        ('refresh_hz', Int64),
        ('battery_wh', Int64),
        ('charger_watts', Int64),
        ('psu_watts', Int64),
        ('wifi', String),
        ('bluetooth', Float64),
        ('weight_kg', Float64),
        ('warranty_months', 

In [14]:
df.select(
    pl.col('brand')
).group_by(
    'brand'
).len().sort(
    by='len',
    descending=True
)

brand,len
str,u32
"""Lenovo""",15992
"""HP""",14114
"""Dell""",14005
"""Apple""",11915
"""ASUS""",10159
"""Acer""",9925
"""Samsung""",8066
"""MSI""",7891
"""Gigabyte""",4900
